In [1]:
import gym

In [2]:
env=gym.make('CartPole-v0')

env class Comes with certain important methods/attributes:

-action_space

-observation_space

-reset(): returns initial state of the game and also resets the enviornment(comes back to starting point)

-step()

-render(): pop-up window of the game


In [3]:
env.reset()

array([ 0.04289837,  0.02267138, -0.03820491, -0.0255932 ])

In [4]:
#reset gave 4 variables that means the enviornment can be defined by 4 variables.

In [5]:
for _ in range(1000): #runs for 1000 steps.
    action=env.action_space.sample() #it consists of actions. see below
    env.step(action)
    env.render()
env.close()
env.reset()

C:\Users\verma\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


array([0.02661521, 0.04775303, 0.00654318, 0.02198249])

In [6]:
env.action_space

Discrete(2)

In [7]:
#2 means it has only two actions left and right movement of cartpole. in short it gives how many axtions
#you can perform

In [8]:
env.observation_space #used to represent n-dimensional tensor in gym.

Box(4,)

In [9]:
env.observation_space.shape

(4,)

In [10]:
env.observation_space.shape[0] #no. of observations

4

2.Playing Games with Random Strategy

-Game Episode(Start to end(game over)--->one episode)

-Step() function in more detail

-Game Over?

In [11]:
#We will have multiple game episodes.
#bENEFIT of this is that in the beiginning of the game
#when the agent is new to enviornment that what action needs to be taken. and will learn step by step

In [12]:

for e in range(20): #e-->episode
    #Play 20 episodes
    observation = env.reset()
    for t in range(50):
        env.render()
        action = env.action_space.sample()
        observation,reward,done,other_info=env.step(action) #step return four things written below
        
        if done:
            #Game episode is over
            print("Game Episode :{}/{} High Score :{}".format(e,20,t))
            break
        
env.close()
print("All 20 episodes are over")

Game Episode :0/20 High Score :12
Game Episode :1/20 High Score :16
Game Episode :2/20 High Score :37
Game Episode :3/20 High Score :24
Game Episode :4/20 High Score :17
Game Episode :5/20 High Score :22
Game Episode :6/20 High Score :33
Game Episode :7/20 High Score :14
Game Episode :8/20 High Score :28
Game Episode :9/20 High Score :28
Game Episode :10/20 High Score :15
Game Episode :11/20 High Score :9
Game Episode :12/20 High Score :12
Game Episode :13/20 High Score :31
Game Episode :14/20 High Score :37
Game Episode :15/20 High Score :16
Game Episode :16/20 High Score :17
Game Episode :17/20 High Score :15
Game Episode :18/20 High Score :34
Game Episode :19/20 High Score :10
All 20 episodes are over


Step()

-observation

-reward

-done

-info

3- Q-Learning

Agent Design and Neural Model

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

In [43]:
class Agent:
    def __init__(self,state_size,action_size): #in this game only 4-state size and 2-actions
        self.state_size=state_size
        self.action_size=action_size 
        self.memory=deque(maxlen=2000)
        self.gamma=0.95 #discount factor in RL
        
        #Exploration vs Exploitation Tradeoff
        
        #Exploration: Good in the beginnning--> helps you to try various random things
        #Exploitation: Sample good experience from the past(memory)--->good in the end.
        
        self.epsilon=1.0 #100% random exploration in the beginning.
        self.epsilon_decay=0.995
        self.epsilon_min=0.01
        self.learning_rate=0.001
        self.model=self._create_model()
    
    def _create_model(self):
        model=Sequential()
        model.add(Dense(24,input_dim=4,activation='relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(2,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=0.001)) #kind of linear regression
        
        return model
    
    def remember(self,state,action,reward,next_state,done):
        #remeber past experience
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        #Sampling according Epsilon greedy method
        
        if np.random.rand()<=self.epsilon:
            #Take a random action
            return random.randrange(self.action_size)
        #Ask neural network to give me the most suitable action
        return np.argmax(self.model.predict(state)[0])#[0] because it gives list of list
    
    def train(self,batch_size=32):
        #Training using a replay buffer
        #We have to sample out batch of experiences forom the buffer/memory and feed every expericence one by one to our neural network
        minibatch = random.sample(self.memory,batch_size)
        for experience in minibatch:
            state,action,reward,next_state,done=experience
            
            # to train we need , X,y:state,expected reward(reward is the bellman equation that we derived)
            if not done:
                #game is not yet over, use bellman equation to apprx the target_Value of reward
                target=reward+self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target=reward
            
            target_f=self.model.predict(state) 
            target_f[0][action]=target
            
            #X=state,Y=target_f
            self.model.fit(state,target_f,epochs=1,verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon*=self.epsilon_decay 
            
    def load(self,name):
        self.model.load_weights(name)
    
    def save(self,name):
        self.model.save_weights(name)

In [44]:
n_episodes=1000
output_dir="cartpole_model/"

In [45]:
agent=Agent(state_size=4,action_size=2)
done=False
state_size=4
action_size=2

In [46]:
for e in range(n_episodes):
    state=env.reset()
    state=np.reshape(state,[1,state_size])
    batch_size=32
    for time in range(500):
        env.render()
        action=agent.act(state) #action is going to be left or rigth (0 or 1)
        next_state,reward,done,other_info=env.step(action)
        
        reward=reward if not done else -10 #randomly-->-10
        next_state=np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done) #Experience for the agent
        
        if done:
            print("Game Episode :{}/{} High Score :{} Exploration Rate:{:.2}".format(e,20,t,agent.epsilon))
            break
        
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    if e%50==0:
        agent.save(output_dir+"weights_"+'{:04d}'.format(e)+".hdf5")

    
print("Deep Q-learner Model Trained")
env.close()
        
        
        

Game Episode :0/20 High Score :10 Exploration Rate:1.0
Game Episode :1/20 High Score :10 Exploration Rate:1.0
Game Episode :2/20 High Score :10 Exploration Rate:0.99
Game Episode :3/20 High Score :10 Exploration Rate:0.99
Game Episode :4/20 High Score :10 Exploration Rate:0.99
Game Episode :5/20 High Score :10 Exploration Rate:0.98
Game Episode :6/20 High Score :10 Exploration Rate:0.98
Game Episode :7/20 High Score :10 Exploration Rate:0.97
Game Episode :8/20 High Score :10 Exploration Rate:0.97
Game Episode :9/20 High Score :10 Exploration Rate:0.96
Game Episode :10/20 High Score :10 Exploration Rate:0.96
Game Episode :11/20 High Score :10 Exploration Rate:0.95
Game Episode :12/20 High Score :10 Exploration Rate:0.95
Game Episode :13/20 High Score :10 Exploration Rate:0.94
Game Episode :14/20 High Score :10 Exploration Rate:0.94
Game Episode :15/20 High Score :10 Exploration Rate:0.93
Game Episode :16/20 High Score :10 Exploration Rate:0.93
Game Episode :17/20 High Score :10 Explorat

KeyboardInterrupt: 

In [1]:
x=np.random.rand(1,4)
model.predict(x) #this gives reward for the state given, there are 2 preddictions only left and right movement.

NameError: name 'np' is not defined

In [2]:
[n for n in range(10) if n%2]

[1, 3, 5, 7, 9]

In [3]:
def my_func(n1,n2):
    return n1+n2

In [5]:
my_func(1,2,3)

NameError: name 'my_func' is not defined